## Простая модель с ручной обработкой

In [ ]:
def sentiment_ML(review: str):
    "функция обрабатывает текст и предсказывает на ML-модели"
    
    clean_tokenized_review = tokenize(review)
    tfid_representation = tfid.transform([clean_tokenized_review]) # ревью состоит из векторов, сверху загружаем vectorizer
    
    pred_proba = model_ml.predict_proba(tfid_representation)
    
    return pred_proba

In [ ]:
# всё, что нужно для модели

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# Возможно нужны будут
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

# Кэшируем стоп-слова
sw = stopwords.words('english')

#Загружаем vectorizer
tfid = load('tfidf.joblib')

#Загружаем модель для sentiment_ML
model_ml = load('logreg.joblib')

def clean(text):
    "Функция чистит текст"
    text = text.lower() #нижний регистр
    text = re.sub('<.*?>', '', text) # Remove HTML from text
    text = text.translate(str.maketrans('', '', string.punctuation)) # удаляем знаки препинания
    text = re.sub(r'\d+', ' ', text) # удаляем числа
    
    return text

def tokenize_review(review: str):
    "Функция чистит текст и возвращает токенизированные слова"
    cleaned_review = clean(review) # cleaning text
    wn_lemmatizer = WordNetLemmatizer() #lemmatization
    reg_tokenizer = RegexpTokenizer('\w+') #tokenization
    
    lemmatized_review = ' '.join([wn_lemmatizer.lemmatize(word, tag[0].lower())\ # лемматизация с учётом части речи
                                if tag[0].lower() in ['a','n','v']\ # word - проверка на adv, noun и verb
                                else wn_lemmatizer.lemmatize(word)\ # простая лемматизация
                                for word, tag in nltk.pos_tag(cleaned_review.split())]) # pos_tag - определяет часть речи
    
    tokenized_review = reg_tokenizer.tokenize_sents([lemmatized_review]) # ревью состоит из токенов
    clean_tokenized_review = ' '.join([word for word in tokenized_review[0] if word not in sw]) # удаляем стоп-слова, сверху объявили sw 
    
    return clean_tokenized_review

## Хорошая модель с ручной обработкой

In [ ]:
def sentiment_RNN(review: str):
    "функция обрабатывает текст и предсказывает на RNN-модели"
    
    clean_tokenized_review = tokenize_review(review)
    
    num_review = [] # здесь будут вместо слов числа из словаря vocab_to_int #векторизуем
    for word in clean_tokenized_review.split(): 
        try:
            num_review.append(vocab_to_int[word]) 
        except KeyError as e:
            print(f'Word {word} not in dictionary!')
            
    #padding 
    padding_review = num_review[-200:] #обрубаем если больше 200 слов
    if len(num_review) <= 200:
        padding_review = list(np.zeros(200 - len(r))) + num_review #дополняем нулями если меньше 200 слов
        
    tensor_review = torch.Tensor(pading_review).long().unsqueeze(0) #создаем тензор ревью для модели
    test_h = model_rnn.init_hidden(1) #создаем hidden_state

    pred = model_rnn(tensor_review, test_h) 
    pred_proba = pred[0].item()
    
    return pred_proba

In [ ]:
# всё что нужно для модели


#загружаем словарь слов с
with open('dict.json', 'r') as fp:
    vocab_to_int = json.load(fp)
    


## Простая модель с обработкой от BERT

In [ ]:
def sentiment_BERT_ML(review: str):
    "функция обрабатывает текст с помощью BERT и предсказывает на ML-модели"
    
    
    return answer_text